In [2]:
!pip install faiss-cpu 

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      /tmp/pip-build-env-kuq2sktd/overlay/lib/python3.10/site-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/lat

In [1]:
import json
import faiss
import numpy as np
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Embedding Metadata
with open('clothing_embedding.json', 'r') as f:
    metadata = json.load(f)

# Load FAISS Index
index = faiss.read_index('clothing_faiss.index')

# Image Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Model for Embedding
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()
model = model.to(device)
model.eval()


def get_image_embedding(img_path):
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = model(image).cpu().numpy().flatten()
    return emb.astype('float32')


def search_similar_images(query_path, k=5):
    query_emb = get_image_embedding(query_path)
    D, I = index.search(query_emb.reshape(1, -1), k)
    
    print(f"Query Image: {query_path}")
    show_images([query_path] + [metadata[i]['image_path'] for i in I[0]])


def show_images(img_paths):
    plt.figure(figsize=(15, 5))
    for idx, path in enumerate(img_paths):
        img = Image.open(path)
        plt.subplot(1, len(img_paths), idx+1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

search_similar_images('clothes/train/pants/01_pants.jpg', k=5)


ModuleNotFoundError: No module named 'faiss'

In [ ]:
import json
import faiss
import numpy as np
import torch
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Embedding Metadata
with open('clothing_embedding.json', 'r') as f:
    metadata = json.load(f)

# Load FAISS Index
index = faiss.read_index('clothing_faiss.index')

# Image Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Model for Embedding
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()
model = model.to(device)
model.eval()


def get_image_embedding(img_path):
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = model(image).cpu().numpy().flatten()
    return emb.astype('float32')


def compute_top_k_accuracy(k=5):
    correct = 0
    total = 0

    for entry in tqdm(metadata, desc="Evaluating"):
        true_category = entry["category"]
        img_path = entry["image_path"]
        
        # Get query image embedding
        query_emb = get_image_embedding(img_path)

        # Search in FAISS index
        D, I = index.search(query_emb.reshape(1, -1), k)

        # Check if any of the top-K results match the true category
        retrieved_categories = [metadata[i]['category'] for i in I[0]]
        if true_category in retrieved_categories:
            correct += 1
        total += 1

    accuracy = (correct / total) * 100
    print(f"Top-{k} Accuracy: {accuracy:.2f}%")
    

# Compute Top-5 Accuracy
compute_top_k_accuracy(k=5)

# Top-1 Accuracy
compute_top_k_accuracy(k=1)


In [ ]:
def mean_average_precision(k=5):
    average_precisions = []
    for entry in tqdm(metadata, desc="Evaluating mAP"):
        true_category = entry["category"]
        img_path = entry["image_path"]
        
        query_emb = get_image_embedding(img_path)
        D, I = index.search(query_emb.reshape(1, -1), k)
        
        retrieved_categories = [metadata[i]['category'] for i in I[0]]
        
        # Calculate precision at each rank position
        relevant_retrievals = [1 if retrieved_categories[i] == true_category else 0 for i in range(k)]
        precision_at_k = np.cumsum(relevant_retrievals) / (np.arange(k) + 1)
        
        # Average precision for this query
        average_precision = np.sum(precision_at_k * relevant_retrievals) / np.sum(relevant_retrievals) if np.sum(relevant_retrievals) > 0 else 0
        average_precisions.append(average_precision)
    
    mAP = np.mean(average_precisions)
    print(f"Mean Average Precision (mAP) at {k}: {mAP:.4f}")

mean_average_precision(k=5)


In [ ]:
def recall_at_k(k=5):
    correct_retrievals = 0
    total_relevant = 0
    
    for entry in tqdm(metadata, desc="Evaluating Recall"):
        true_category = entry["category"]
        img_path = entry["image_path"]
        
        query_emb = get_image_embedding(img_path)
        D, I = index.search(query_emb.reshape(1, -1), k)
        
        retrieved_categories = [metadata[i]['category'] for i in I[0]]
        
        # Count how many of the relevant items (same category) are in the top K
        relevant_retrievals = [1 if retrieved_categories[i] == true_category else 0 for i in range(k)]
        correct_retrievals += np.sum(relevant_retrievals)
        total_relevant += 1  # Every image has at least one relevant category (itself)
    
    recall = correct_retrievals / total_relevant
    print(f"Recall at {k}: {recall:.4f}")

recall_at_k(k=5)


In [ ]:
def precision_at_k(k=5):
    total_precision = 0
    
    for entry in tqdm(metadata, desc="Evaluating Precision"):
        true_category = entry["category"]
        img_path = entry["image_path"]
        
        query_emb = get_image_embedding(img_path)
        D, I = index.search(query_emb.reshape(1, -1), k)
        
        retrieved_categories = [metadata[i]['category'] for i in I[0]]
        
        # Count how many of the retrieved items are relevant (same category)
        relevant_retrievals = [1 if retrieved_categories[i] == true_category else 0 for i in range(k)]
        
        # Precision for this query
        precision_at_k = np.sum(relevant_retrievals) / k
        total_precision += precision_at_k
    
    avg_precision = total_precision / len(metadata)
    print(f"Average Precision at {k}: {avg_precision:.4f}")

precision_at_k(k=5)


In [ ]:
import time

def evaluate_search_speed(num_queries=100, k=5):
    total_time = 0
    
    for entry in tqdm(metadata[:num_queries], desc="Evaluating Search Speed"):
        img_path = entry["image_path"]
        query_emb = get_image_embedding(img_path)
        
        start_time = time.time()
        D, I = index.search(query_emb.reshape(1, -1), k)
        total_time += time.time() - start_time
    
    avg_search_time = total_time / num_queries
    print(f"Average Search Time for {num_queries} queries: {avg_search_time:.4f} seconds")

# Example usage
evaluate_search_speed(num_queries=100, k=5)


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from tqdm import tqdm

def visualize_embeddings():
    embeddings = []
    labels = []
    
    # Map categories to numerical values
    category_map = {category: idx for idx, category in enumerate(set(entry['category'] for entry in metadata))}
    
    for entry in tqdm(metadata, desc="Collecting Embeddings"):
        img_path = entry["image_path"]
        embedding = get_image_embedding(img_path)
        embeddings.append(embedding)
        labels.append(category_map[entry["category"]])  # Store numerical label
    
    # Reduce dimensionality with PCA
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(np.array(embeddings))
    
    plt.figure(figsize=(10, 10))
    
    # Use the 'viridis' colormap, but you can choose another colormap (e.g., 'tab20')
    scatter = plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap='tab20', alpha=0.6)
    
    # Create a legend using category names
    handles, _ = scatter.legend_elements()
    plt.legend(handles, category_map.keys(), title="Categories")
    
    plt.show()

visualize_embeddings()
